<a href="https://colab.research.google.com/github/jackl14/Carbon-Footprint-Huella-de-Carbono/blob/main/CO2CalculatorEnglish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

xls = pd.ExcelFile('ICE DB V3.0 - 10 Nov 2019.xlsx')

def get_value_to_right(df, column_name, name):
    try:
        # Find the index of the row where 'Name' matches the given name
        index = df.index[df['Unnamed: 5'] == name][0]

        # Get the value in the column to the right
        value_to_right = df.loc[index, column_name]

        return value_to_right
    except IndexError:
        return None  # Handle case where name is not found in dataframe

def remove_entries_by_value(value, list1, list2, list3):
    try:
        idx = list1.index(value)  # Find index of 'value' in list1
        del list1[idx]            # Remove entry from list1 at index idx
        del list2[idx]            # Remove entry from list2 at index idx
        del list3[idx]            # Remove entry from list3 at index idx
    except ValueError:
        print(f"'{value}' not found in {list1}.")

def select_material():
  #Choose material
  sheet_names = xls.sheet_names
  sheet_list = []
  for entry in sheet_names:
    if entry != 'Concrete':
      sheet_list.append(entry)
  materials = sheet_list[10:19]
  print("Here are the possible material options: ")
  for entry in materials:
    print(entry)
  selection = input("Enter the material you wish to add: ")
  if selection not in materials:
    print("You did not enter a valid option, please try again.")
    select_material()
  else:
    return selection

#select_material()

def search_material(selection):

  df_raw = pd.read_excel('ICE DB V3.0 - 10 Nov 2019.xlsx', sheet_name=selection)
  df = df_raw.fillna("Empty")

  for i in range(len(df)):
    location = df.loc[i,"Unnamed: 5"]
    if location == '<<< Start of Main Embodied Carbon Data >>>':
      start = i
    elif location == '<<< End of Main Embodied Carbon Data >>>':
      endpt = i

  list_of_interest = df['Unnamed: 5'][start:endpt].tolist()

  clean_list = []
  keep_going = True
  i = 2

  while keep_going:
      point = list_of_interest[i]
      #print(point)
      if point == "Empty" and all(item == "Empty" for item in list_of_interest[i+1:i+4]):
          break  # Break the loop if current and next three entries are all "Empty"
      clean_list.append(point)
      i += 1

  final_list = []

  if selection == 'Aggregates_Sand':
    words = ['Aggregates and sand']
  elif selection == 'Bitumen':
    words = ["Bitumen", "bitumen"]
  elif selection == "Cement_and_Mortar":
    words = ["CEM", "Mortar ", "admixtures"]
  elif selection == 'Clay_Bricks':
    words = ['Clay']
  else:
    words = [selection]

  for word in words:
    for item in clean_list:
      if word in str(item):
        final_list.append(item)

  unique_final = []
  [unique_final.append(x) for x in final_list if x not in unique_final]


  for i in unique_final:
    print(i)

  try:
    material_selection = input("Enter the material you wish to use:")
    carbon_footprint_str = get_value_to_right(df, 'Unnamed: 6', material_selection)
    carbon_footprint = float(carbon_footprint_str)
    return material_selection, carbon_footprint

  except Exception as e:
    print("You seem to have entered something improperly")

#search_material('Cement_and_Mortar')


def add_material():
  print("Here are the options:")
  print()
  selection = select_material()
  print()
  print("Now choose the specific type. Here are the options:")
  print()
  try:
    material, footprint = search_material(selection)
  except:
    "Carbon emmisions for your entry could not be found. Perhaps try a similar material."
  print()
  print("Now enter the quantity of the material that you wish to use. Assume kg unless otherwise specified.")
  quantity_str = input("Quantity: ")
  try:
    quantity = float(quantity_str)
    print()
    emmisions = footprint*quantity
    print(f'The total embodied emmisions for {quantity} units of {material} is {emmisions} kg of CO2.')
    return material, emmisions, quantity
  except:
    print("You did not enter a valid number")

def main():
  print("Welcome to the project carbon emmision calculator tool.")
  print()
  materials = []
  emmisions = []
  quantities = []

  while True:
    print()
    print("To add another material, enter a.")
    print("To see a current summary for your project, enter r.")
    print("To remove a material, enter x.")
    print("To stop running the tool, enter d.")
    print()
    choice = input("Your choice: ")
    print()

    if choice == 'a':

      material, emmision, quantity = add_material()
      materials.append(material)
      emmisions.append(emmision)
      quantities.append(quantity)

    elif choice == 'r':

      df = pd.DataFrame({
      'Material': materials,
      'Quantity (kg unless otherwise specified)': quantities,
      'Emmisions (kg CO2)': emmisions
        })
      # Set 'Material' as the index
      df.set_index('Material', inplace=True)

      # Calculate totals for integer columns
      totals = {}
      for col in df.columns:
          if pd.api.types.is_numeric_dtype(df[col]):  # Check if column is numeric (integer or float)
              totals[col] = df[col].sum()

      # Convert totals to DataFrame row
      totals_df = pd.DataFrame(totals, index=['Total'])

      # Concatenate totals row to the original DataFrame
      df = pd.concat([df, totals_df])
      print(df)

      save = input("Do you wish to dowload this spreadsheet to your computer? (y/n): ")
      if save == 'y':

        name = input("What do you want to call this file? ")
        # Save DataFrame to CSV in Colab
        csv_file_path = '/content/sample_data/' + name + '.csv'  # Change path as needed
        df.to_csv(csv_file_path, index=False)

        # Download CSV file from Colab
        from google.colab import files
        files.download(csv_file_path)

    elif choice == 'x':
      for item in materials:
        print(item)
      print()
      removed = input("Enter the name of the item that you wish to remove: ")
      remove_entries_by_value(removed, materials, quantities)

    elif choice == 'd':
      break

main()

Welcome to the project carbon emmision calculator tool.


To add another material, enter a.
To see a current summary for your project, enter r.
To remove a material, enter x.
To stop running the tool, enter d.

Your choice: a

Here are the options:

Here are the possible material options: 
Aggregates_Sand
Aluminium
Asphalt
Bitumen
Cement_and_Mortar
Clay_Bricks
Glass
Steel
Timber
Enter the material you wish to add: Glass

Now choose the specific type. Here are the options:



/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Glass, General, per kg
Glass, Glazing, Double, per kg
Glass, Glazing triple, per kg
Glass, Toughened, per kg
Glass, Multi layer safety, filled core, fire resistant, toughened, per kg
Glass, Multi layer safety, unfilled, per kg
Glass, sky light or roof, with frame, per kg
Enter the material you wish to use:Glass, General, per kg

Now enter the quantity of the material that you wish to use. Assume kg unless otherwise specified.
Quantity: 599

The total embodied emmisions for 599.0 units of Glass, General, per kg is 860.7432712459564 kg of CO2.

To add another material, enter a.
To see a current summary for your project, enter r.
To remove a material, enter x.
To stop running the tool, enter d.

Your choice: a

Here are the options:

Here are the possible material options: 
Aggregates_Sand
Aluminium
Asphalt
Bitumen
Cement_and_Mortar
Clay_Bricks
Glass
Steel
Timber
Enter the material you wish to add: Steel

Now choose the specific type. Here are the options:



/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Steel, UO Pipe
Steel, Tin-free Electrolytic Chrome Coated Steel Sheet - Tin-free (ECCS)
Steel, electrogalvanized steel
Steel, welded pipe
Steel, Organic coated sheet
Steel, Tinplate
Steel, finished cold-rolled coil
Steel, hot-dip galvanized steel
Steel, Plate
Steel, pickled hot-rolled coil
Steel, Wire rod
Steel, Hot Rolled Coil
Steel, Rebar
Steel, Section
Steel, Engineering steel
Steel, global seamless tube
Further Information - Recycling and Recyclability Benefits of Steel
Environmental Value of Scrap Steel
Enter the material you wish to use:Steel, Rebar

Now enter the quantity of the material that you wish to use. Assume kg unless otherwise specified.
Quantity: 300

The total embodied emmisions for 300.0 units of Steel, Rebar is 597.0 kg of CO2.

To add another material, enter a.
To see a current summary for your project, enter r.
To remove a material, enter x.
To stop running the tool, enter d.

Your choice: r

                        Quantity (kg unless otherwise specified)  \
Glas